In [1]:
# Run this cell first!
class _CgAnswers: ...
answers = _CgAnswers()


# BScBAN Y2 - Algorithms and Data Structure - Assignment 2

___


This assignment is made for auto-grading with CodeGrade in Canvas
* Write your code
* **NEVER TOUCH THE CODE WHERE IT IS MARKED "DO NOT CHANGE"**
* Submit through Canvas
* Check the feedback

___

This assignment makes use of material covered in Weeks 1 and 2:
* Class in general
* Class attributes and methods
* Class inheritance
* Method overload
* Abstract methods


___

* There are 2 questions, for a total of 40 points:
  * Q1: 20 points 
  * Q2: 20 points 
  
* Q1 and Q2 are independent

___




# Question 1 - 20 points

This question will refer to a problem of urban planification.

Consider a neighborhood as grid of N rows and M columns, in the typical way of US suburban development.

There are N avenues crossed by M streets. Each square on the grid represents a block of buildings.

Our problem is the following:
* Given the types of buildings at our disposal: Housing, Commercial, Hospital, Office, Park
* Given some distance constraints: proximity between Housing and Park, distance between Housing and Hospital
* Given some population constraints: at least X inhabitants, Y commercial areas, etc...
* We want to optimize the allocation of the neighborhood blocks

We receive different scenarios for allocation, each scenario being a file with the following information:
* a description of the grid
* a list of blocks with their type and location


## Test Code

This code will build a scenario and save it to file:

```python
# Create Data
grid = Grid(rows=5, cols=5)

h1 = Housing(name="Dense Housing 1", x=2, y=2, dense=True, grid=grid)
h2 = Housing(name="Sparse Housing 1", x=3, y=4, dense=False, grid=grid)

o1 = Office(name="Office 1", x=0, y=2, grid=grid)
o2 = Office(name="Office 2", x=4, y=0, grid=grid)

p1 = Park(name="Park 1", x=1, y=1, trees=True, grid=grid)
p2 = Park(name="Park 2", x=3, y=3, trees=False, grid=grid)

# TEST
print(grid.allocated == 6)  # expect True (1)
print(grid.free == 19)      # expect True (2)
print(grid[(2, 2)] is h1)   # expect True (3)
print(grid[(0, 0)] is None) # expect True (4)
print(Block.count == 6)     # expect True (5)
print(Block.directory["Park 1"] is p1)  # expect True  (6)
print(str(h1) == "Housing,Dense Housing 1,2,2,True") # expect True (7)
print(str(grid) == "Grid,5,5")   # expect True (8)
print(p1.trees is True)     # expect True (9)
print(p1.grid is grid)      # expect True (10)

h3 = Housing.from_string("Housing,Skyscrapers,2,4,True", grid=grid)
print(h3.x == 2)  # expect True (11)
print(grid[(2, 4)] is h3)  # expect True (12)

o3 = Office.from_string("Office,Luxury Offices,4,4", grid=grid)
print(o3.y == 4)           # expect True (13)
print(grid[(4, 4)] is o3)  # expect True (14)

p3 = Park.from_string("Park,St James Park,1,3,True", grid=grid)
print(p3.trees is True)             # expect True (15)
print(p3.name == "St James Park")   # expect True (16)
print(grid[(1, 3)] is p3)           # expect True (17)

print(grid.allocated == 9)          # expect True (18)
print(grid[(4, 1)] is None)         # expect True (19)

grid.save("scenario_01.txt")

with open("scenario_01.txt", "r") as src:
    lines = src.readlines()

reference = """Grid,5,5
Housing,Dense Housing 1,2,2,True
Housing,Sparse Housing 1,3,4,False
Housing,Skyscrapers,2,4,True
Office,Office 1,0,2
Office,Office 2,4,0
Office,Luxury Offices,4,4
Park,Park 1,1,1,True
Park,Park 2,3,3,False
Park,St James Park,1,3,True""".splitlines()

print(
    len(lines) == len(reference) and
    all(l.strip() in reference for l in lines)
)   # expect True (20)
```

The file `scenario_01.txt` should have the following contents (maybe not in the same order):
```
Grid,5,5
Housing,Dense Housing 1,2,2,True
Housing,Sparse Housing 1,3,4,False
Housing,Skyscrapers,2,4,True
Office,Office 1,0,2
Office,Office 2,4,0
Office,Luxury Offices,4,4
Park,Park 1,1,1,True
Park,Park 2,3,3,False
Park,St James Park,1,3,True
```

## Modelisation

From the code above, infer the classes, attributes, methods that you need:
* **CLASSES** and their hierarchy
  * Block
    * Housing
    * Office
    * Park
  * Grid
* **ATTRIBUTES**
  * Block objects (and sub-classes objects) have:
    * name
    * x
    * y
    * grid
  * Block class has:
    * count (number of Block objects created)
    * directory (dictionary of block objects by name)
  * Housing objects:
    * dense
  * Park objects:
    * trees
  * Grid objects:
    * rows
    * cols
    * allocated (number of allocated blocks)
    * free (number of free blocks)
    * a storage of where all blocks are located (see hint)
* **METHODS**
  * all classes implement `str(x)` 
  * Grid implements `grid[(x, y)]` to get the block located at coordinates `x, y`. If no block is at these coordinates, it should return `None`
  * Grid objects have a method `add_block` so that an object of class Block can be added to the grid. This method takes care that attributes `allocated` and `free` are updated
  * Grid objects have a method `save` to save the grid into a file (hint: make use of `str(x)` to get the right text for the blocks in the grid)
  * Sub-classes of Block implement a classmethod `from_string`


## Hints

Dictionaries are very convenient:

```python
things = {}   # create empty dictionary
things[(12, 3)] = "Ring"

print(things[(12, 3)])  # "Ring"
print((12, 3) in things)  # True
print((0, 0) in things)   # False
```

## Instructions / Grading

* Write the code for all the classes in ONE single cell
* Place the test code in another single cell below 
* Run the cell with class definition, then the cell with test code
* See the result, there will be "True" or "False", 20 of them.
* Every "False" is the indication that something needs to be debugged
* Change your code, Run the cell with classes definitions, then run the test code again
* You will receive 1 point per "True"
* CodeGrade will use the same code, with different numbers / names
* CodeGrade will display "No answer given" when any of the classes `Grid`, `Block`, `Housing`, `Office`, `Park` is not defined

In [2]:
from types import ClassMethodDescriptorType
from os import name
### BEGIN SOLUTION
import numpy as np

class Grid:
    def __init__(self, rows:int, cols:int):
      self.rows = rows
      self.cols = cols 
      self.allocated=0
      self.free=0
      self.storage={}

    def add_block(self, block:"Block"): 
      self.storage[(block.x,block.y)]=block
      self.allocated+= 1
      self.free = self.rows*self.cols - self.allocated
      return self.storage
    def __getitem__(self, value):
      if (value in self.storage):
        return self.storage[value]
      else:
        return None
    def __str__(self):
      string= f"Grid,{self.rows},{self.cols}"
      return string
    def save(self, title):
      file1=open(title,"w")
      file1.write(str(self))
      for i in self.storage.values():
        file1.write('\n')
        file1.write(str(i))
      file1.close()


class Block:
  count=0
  directory={}
  def __init__(self, name: str, x: int,y: int, grid:Grid):
    self.name=name
    self.x=x
    self.y=y
    self.grid=grid
    Block.count += 1
    grid.add_block(self)

  

class Housing(Block):
  def __init__(self, name:str, x: int,y: int, grid: Grid, dense: bool):
    super().__init__(name=name,x=x,y=y,grid=grid)
    self.dense=dense
    Block.directory[self.name]=self

  @classmethod
  def from_string(self,m:str,grid)->"Housing":
    split=m.split(",")
    return Housing(name=str(split[1]),x=int(split[2]),y=int(split[3]),dense=split[4],grid=grid)
  def __str__(self):
    string= f"Housing,{self.name},{self.x},{self.y},{self.dense}"
    return string

class Office(Block):
  def __init__(self, name:str, x: int,y: int, grid: Grid):
    super().__init__(name=name,x=x,y=y,grid=grid)
    Block.directory[self.name]=self
  @classmethod
  def from_string(self,m:str,grid)->"Office":
    split=m.split(",")
    return Office(name=str(split[1]),x=int(split[2]),y=int(split[3]),grid=grid)
  def __str__(self):
    string= f"Office,{self.name},{self.x},{self.y}"
    return string

class Park(Block):
  def __init__(self, name:str, x: int,y: int, trees: bool,grid: Grid):
    super().__init__(name=name,x=x,y=y,grid=grid)
    self.trees=trees
    Block.directory[self.name]=self
  @classmethod
  def from_string(self,m:str,grid)->"Park":
    split=m.split(",")
    return Park(name=str(split[1]),x=int(split[2]),y=int(split[3]),trees=bool(split[4]),grid=grid)
  def __str__(self):
    string= f"Park,{self.name},{self.x},{self.y},{self.trees}"
    return string

### END SOLUTION



In [3]:
# Create Data
grid = Grid(rows=5, cols=5)

h1 = Housing(name="Dense Housing 1", x=2, y=2, dense=True, grid=grid)
h2 = Housing(name="Sparse Housing 1", x=3, y=4, dense=False, grid=grid)

o1 = Office(name="Office 1", x=0, y=2, grid=grid)
o2 = Office(name="Office 2", x=4, y=0, grid=grid)

p1 = Park(name="Park 1", x=1, y=1, trees=True, grid=grid)
p2 = Park(name="Park 2", x=3, y=3, trees=False, grid=grid)

# TEST
print(grid.allocated == 6)  # expect True (1)
print(grid.free == 19)      # expect True (2)
print(grid[(2, 2)] is h1)   # expect True (3)
print(grid[(0, 0)] is None) # expect True (4)
print(Block.count == 6)     # expect True (5)
print(Block.directory["Park 1"] is p1)  # expect True  (6)
print(str(h1) == "Housing,Dense Housing 1,2,2,True") # expect True (7)
print(str(grid) == "Grid,5,5")   # expect True (8)
print(p1.trees is True)     # expect True (9)
print(p1.grid is grid)      # expect True (10)

h3 = Housing.from_string("Housing,Skyscrapers,2,4,True", grid=grid)
print(h3.x == 2)  # expect True (11)
print(grid[(2, 4)] is h3)  # expect True (12)

o3 = Office.from_string("Office,Luxury Offices,4,4", grid=grid)
print(o3.y == 4)           # expect True (13)
print(grid[(4, 4)] is o3)  # expect True (14)

p3 = Park.from_string("Park,St James Park,1,3,True", grid=grid)
print(p3.trees is True)             # expect True (15)
print(p3.name == "St James Park")   # expect True (16)
print(grid[(1, 3)] is p3)           # expect True (17)

print(grid.allocated == 9)          # expect True (18)
print(grid[(4, 1)] is None)         # expect True (19)

grid.save("scenario_01.txt")

with open("scenario_01.txt", "r") as src:
    lines = src.readlines()

reference = """Grid,5,5
Housing,Dense Housing 1,2,2,True
Housing,Sparse Housing 1,3,4,False
Housing,Skyscrapers,2,4,True
Office,Office 1,0,2
Office,Office 2,4,0
Office,Luxury Offices,4,4
Park,Park 1,1,1,True
Park,Park 2,3,3,False
Park,St James Park,1,3,True""".splitlines()

print(
    len(lines) == len(reference) and
    all(l.strip() in reference for l in lines)
)   # expect True (20)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [4]:
# DO NOT CHANGE 
answers.Q1 = {"Block": Block, "Housing": Housing, "Office": Office, "Park": Park, "Grid": Grid}

___

# Question 2 - 20 points

In this question, we will work with projects and Net Present Value (NPV) of cash-flows (see your course of Y1 Finance.)

**Reminder**:
* Given projected cash-flows $C_0$ in Year 0, $C_1$ in Year 1, ..., $C_N$ in Year N
* Given a discount rate $r$

$NPV = \sum_{k=0}^N \frac{C_k}{(1 + r)^k} = C_0 + \frac{C_1}{1+r} + \frac{C_2}{(1+r)^2} + ...$

We consider different types of Cash-Flows:
* One Time: happening on a specific year
* Growing: starting on a specific year, ending on a year, an initial value, a yearly growth rate
* Growing Perpetuity: starting with a specific year, an initial value, a yearly growth rate

For example, we want to sell tickets for customers to travel to the moon:
* Year 0
  * Development Phase
  * One-Time cash flow of $-1B\$$
* Year 1
  * Development Phase
  * One-Time cash flow of $-2B\$$
* Year 2-3-4
  * First sales, no dev costs
  * Growing 
  * Initial value: $100M\$$
  * Growth rate: $20\%$
* Year 5 to infinite
  * Standard production
  * Growing perpetuity
  * Initial value: $+200M\$$
  * Growing rate: $2.5\%$

The IRR of the project is the discount rate for which the NPV is 0.

We consider the following approach:
* Cash-Flows can be:
  * single cash-flows: in year 0, we expect a cash-flow of $-1B\$$
  * growing cash-flow: starting year 2, up to year 4, we have cash-flows of `100, 120, 144`
  * growing perpetuity: forever starting with year 5, we have a growing cash-flow
* We can check whether a specific year is included in a cash-flow (e.g. year 3 is included in the growing cash-flow, but not year 6)
* Project is made of a schedule of cash-flows 
* A cash flow can be added to a project: it modified the schedule of the project. It is not a new project.
* 2 projects can be merged into 1, combining the schedules. It is a new project.
* Projects have:
  * a schedule
  * a NPV (given a discount rate)
  * an IRR (obtained by locating for which discount rate does the NPV turn to 0)

Here is the testing code:

```python
moon = Project(name="Moon")
keep_moon = moon

year_0 = OneTime(year=0, cash=-1e9)
year_1 = OneTime(year=1, cash=-2e9)
launch = Growing(year_start=2, year_end=4, cash_start=1e8, g=0.2)
perpetuity = GrowingPerpetuity(year_start=5, cash_start=2e8, g=0.025)

# Checking that we have abstract methods and inheritance
import inspect
print(inspect.isabstract(CashFlow) and all(x in CashFlow.__abstractmethods__ for x in ["__contains__", "__str__", "discount"]))  # expect True (1)
print(isinstance(launch, CashFlow))   # expect True (2)
print(isinstance(perpetuity, CashFlow)) # expect True (3)

print(3 in launch)      # expect True (4)
print(2 not in year_1)  # expect True (5)

# cash-flows are always discounted to Year 0
print(abs(year_1.discount(r=0.05) - (-1904761904.7619047)) < 1)    # expect True (6)
print(abs(launch.discount(r=0.05) - 312832616.03961307) < 1)   # expect True (7)
print(abs(perpetuity.discount(r=0.05) - 6581619798.335054) < 1) # expect True (8)

flows = [year_0, year_1, launch, perpetuity]

for f in flows:
    moon += f

print(moon.schedule_count == 4)    # expect True (9)
print(abs(moon.npv(r=0.05) - 3989690509.612763) < 1)    # expect True (10)
print(abs(moon.npv(r=0.1) - (-725656262.0950305)) < 1)   # expect True  (11)
print(abs(moon.irr(scan_from=0.05, scan_to=0.1, epsilon=1e-3) - 0.082) < 0.001)     # expect True (12)

print(str(moon) == "Project Moon - IRR [8% - 9%]") # expect True (13)

print(len(moon[4]) == 1)     # expect True (14)
print(moon[4][0] is launch)  # expect True (15)

extra_dev = OneTime(year=3, cash=-5e8)
moon += extra_dev

print(str(moon) == "Project Moon - IRR [7% - 8%]") # expect True (16)
print(moon is keep_moon)  # expect True(17)
print(len(moon[3]) == 2 and all(x in moon[3] for x in [launch, extra_dev]))  # expect True (18)

mars = Project("Mars")
mars_y0 = OneTime(year=0, cash=-4e9)
mars_y1 = OneTime(year=1, cash=-4e9)
mars_y2 = OneTime(year=2, cash=-4e9)
mars_ops = GrowingPerpetuity(year_start=3, cash_start=1e8, g=0.03)

mars_cashflows = [mars_y0, mars_y1, mars_y2, mars_ops]

for f in mars_cashflows:
    mars += f

space_portfolio = moon + mars

print(str(space_portfolio) == "Project Moon + Mars - IRR [4% - 5%]")  # expect True (19)
print(len(space_portfolio[3]) == 3 and all(x in space_portfolio[3] for x in [extra_dev, launch, mars_ops]))   # expect True (20)
```

## Modelisation

You will get less hints for this exercise.
* It has to be impossible to create objects of class `CashFlow`
* `CashFlow` makes it mandatory for subclasses to implement `discount` method
* `CashFlow` makes it mandatory for subclasses to implement the operators:
  * `str(cf)`: method `__str__`: the returned string is up to you, it is not tested
  * `3 in cf`: method `__contains__(self, key)`: here `3` is the key. It returns `True` when the cash-flow happens in Year 3. In the code, `3 in launch` returns `True`. `7 in perpetuity` returns `True`.
* Classes `OneTime`, `Growing`, `GrowingPerpetuity` can create objects
  * Their constructor's arguments make sense in Finance
  * The way to compute their NPV at year 0 (method `discount`) is different for each
* `Project` has a schedule: a list of objects `CashFlow` which is not in the constructor parameters
  * the attribute `schedule_count` is the number of objects in this list
* The following operations are supported by `__add__(self, other)`:
  * `project + cashflow`: returns the object `project`, adds the object `cashflow` to the list of cashflows of `project`
  * `project1 + project2` : creates a **NEW** project by merging the 2 projects
    * its name is "name1 + name2", using the names of both projects
    * its schedule is the concatenation of both schedules
    * the `schedule_count` is the sum of both counters
* `Project` has the method `npv`:
  * Gets the NPV of the whole project at Year 0
* `Project` also has the method `irr`
  * Computes the Internal Return Rate
  * See in the code for the arguments
  * Try different values for the discount rate, between a starting value and an ending value, separated by epsilon
  * Return the first value after the sign of the NPV has changed
* `str(project)` displays the project name, along with an approximation of the IRR printed as %
  * use `irr` with a epsilon of 1% 
  * if you find 0.1, then display `[9% - 10%]`
* `project[3]` is supported by `__getitem__(self, index)`, returns the list of cash-flows in the project's schedule for which there is a cash-flow in year 3

In [5]:
from typing import List
from abc import ABC
from abc import abstractmethod
import numpy as np
import math

class CashFlow(ABC):
  #needs to be impossible to make an object 
  #def discount(self, r:float):
  @abstractmethod
  def __init__(self):
    ...
  @abstractmethod
  def __str__(self):
    ...
  @abstractmethod
  def __contains__(self):
    ...
  @abstractmethod
  def discount(self):
    ...



class Project:
  def __init__(self, name:int):
    self.name=name
    self.schedule_count =0
    self.schedule =[]
  def __add__(self,other)->"Project":
    if isinstance(other, CashFlow):
      self.schedule.append(other)
      self.schedule_count +=1
      return self
    new_project=Project(self.name+" + " +other.name)
    new_project.schedule_count=self.schedule_count+other.schedule_count
    new_project.schedule=self.schedule+other.schedule
    return new_project

  def npv(self,r:float):
    npv=0
    for cashflow in self.schedule:
      npv+=cashflow.discount(r)
    return npv

  def irr(self,scan_from, scan_to, epsilon):
    while self.npv(scan_from)>0:
      scan_from+=epsilon
      continue
    else:
      return scan_from
    return irr 

  def __str__(self)->str:
    irr1=self.irr(scan_from=0.04, scan_to=1, epsilon=0.01)
    #lower_value = math.floor(irr1)
    #upper_value  = math.ceil(irr1)
    #string= f"Project",{self.name}," - IRR [",{lower_value},"% - ",{upper_value},"%]"
    irr1 *=100

    string= f"Project {self.name} - IRR [{int((irr1-1))}% - {int(irr1)}%]"
    return string
    
    #return string
  def __getitem__(self,year):
    count=[]
    for cash in self.schedule:
      if year in cash:
        count.append(cash)
    return count


class OneTime(CashFlow):
  def __init__(self, year:int, cash:float):
    self.year=year
    self.cash=cash
  
  def discount(self,r:float):
    PV=self.cash/((r+1)**(self.year))
    return PV

  def __contains__(self,key:int):
    if key == self.year:
      return True
    else:
      return False

  def __str__(self):
    return str(self.year)


class Growing(CashFlow):
  def __init__(self, year_start:int, year_end:int, cash_start:float,g:float):
    self.year_start=year_start
    self.year_end=year_end
    self.cash_start=cash_start
    self.g=g
  def __contains__(self,key:int):
    if key >= self.year_start and key <= self.year_end:
      return True
    else:
      return False 
  def discount(self,r:float): 
    self.r = r
    sv = (self.cash_start / (self.r - self.g)) * (1 - ((1 + self.g)/(1+self.r))**(self.year_end-self.year_start + 1))
    pv = sv / (1 + self.r)** (self.year_start - 1)
    return pv
  def __str__(self):
    return str(self.year_end)
    
class GrowingPerpetuity(CashFlow):
  def __init__(self,year_start:int,cash_start:float,g:float):
    self.year_start=year_start
    self.cash_start=cash_start
    self.g=g
  def __str__(self):
    return str(self.year_start)

  def __contains__(self,key:int):
    if key >= self.year_start:
      return True
    else:
      return False 
  def discount(self,r:float):
    return (self.cash_start/(r-self.g))/(1+r)**(self.year_start-1)


#test code:
#creating objects
moon = Project(name="Moon")
keep_moon = moon
year_0 = OneTime(year=0, cash=-1e9)
year_1 = OneTime(year=1, cash=-2e9)
launch = Growing(year_start=2, year_end=4, cash_start=1e8, g=0.2)
perpetuity = GrowingPerpetuity(year_start=5, cash_start=2e8, g=0.025)

# Checking that we have abstract methods and inheritance
import inspect
print(inspect.isabstract(CashFlow) and all(x in CashFlow.__abstractmethods__ for x in ["__contains__", "__str__", "discount"]))  # expect True (1)
print(isinstance(launch, CashFlow))   # expect True (2)
print(isinstance(perpetuity, CashFlow)) # expect True (3)
print(3 in launch)      # expect True (4)
print(2 not in year_1)  # expect True (5)
# cash-flows are always discounted to Year 0
print(abs(year_1.discount(r=0.05) - (-1904761904.7619047)) < 1)    # expect True (6)
print(abs(launch.discount(r=0.05) - 312832616.03961307) < 1)   # expect True (7)
print(abs(perpetuity.discount(r=0.05) - 6581619798.335054) < 1) # expect True (8)
flows = [year_0, year_1, launch, perpetuity]

for f in flows:
    moon += f

print(moon.schedule_count == 4)    # expect True (9)
print(abs(moon.npv(r=0.05) - 3989690509.612763) < 1)    # expect True (10)
print(abs(moon.npv(r=0.1) - (-725656262.0950305)) < 1)   # expect True  (11)
print(abs(moon.irr(scan_from=0.05, scan_to=0.1, epsilon=1e-3) - 0.082) < 0.001)     # expect True (12)
print(str(moon) == "Project Moon - IRR [8% - 9%]") # expect True (13)

print(len(moon[4]) == 1)     # expect True (14)
print(moon[4][0] is launch)  # expect True (15)

extra_dev = OneTime(year=3, cash=-5e8)
moon += extra_dev

print(str(moon) == "Project Moon - IRR [7% - 8%]") # expect True (16)
print(moon is keep_moon)  # expect True(17)
print(len(moon[3]) == 2 and all(x in moon[3] for x in [launch, extra_dev]))  # expect True (18)

mars = Project("Mars")
mars_y0 = OneTime(year=0, cash=-4e9)
mars_y1 = OneTime(year=1, cash=-4e9)
mars_y2 = OneTime(year=2, cash=-4e9)
mars_ops = GrowingPerpetuity(year_start=3, cash_start=1e8, g=0.03)

mars_cashflows = [mars_y0, mars_y1, mars_y2, mars_ops]

for f in mars_cashflows:
    mars += f

space_portfolio = moon + mars

print(str(space_portfolio) == "Project Moon + Mars - IRR [4% - 5%]")  # expect True (19)
print(len(space_portfolio[3]) == 3 and all(x in space_portfolio[3] for x in [extra_dev, launch, mars_ops]))   # expect True (20)


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [6]:
# DO NOT CHANGE
answers.Q2 = {"Project": Project, "CashFlow": CashFlow, "OneTime": OneTime, "Growing": Growing, "GrowingPerpetuity": GrowingPerpetuity}